In [7]:
pip install ultralytics opencv-python numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 79.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
from dataclasses import dataclass
from typing import Tuple, Optional
!pip install -q "ultralytics==8.3.0"

import cv2
import numpy as np
from ultralytics import YOLO


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.3/881.3 kB 16.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 27.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━

In [7]:
@dataclass
class SideViewFeatures:
    # All scores are 0 (low), 1 (medium), 2 (high)
    ribs_score: int = 0          # ribs visibility (more edges → thinner)
    spine_score: int = 0         # backbone sharpness
    hips_score: int = 0          # hip bone sharpness
    tail_fat_score: int = 0      # fat pads around tail head (smoothness)
    brisket_fat_score: int = 0   # brisket/underline fat (folds/bulge)


In [8]:
class CowDetector:
    """
    Simple YOLOv8 wrapper.
    We just take the highest-confidence bounding box and crop it,
    assuming the main object is the cow.
    """

    def __init__(self, model_path: str = "yolov8n.pt", conf: float = 0.25):
        self.model = YOLO(model_path)
        self.conf = conf

    def detect_and_crop(self, image_bgr: np.ndarray) -> np.ndarray:
        """
        Detect cow and return cropped BGR patch.
        If detection fails, returns the original image.
        """
        # Use model.predict() explicitly on numpy array
        results_list = self.model.predict(
            source=image_bgr,
            conf=self.conf,
            verbose=False
        )

        r = results_list[0]

        # If no boxes, just return original image
        if r.boxes is None or len(r.boxes) == 0:
            return image_bgr

        # Pick highest-confidence box
        best_box = max(r.boxes, key=lambda b: float(b.conf[0]))

        x1, y1, x2, y2 = best_box.xyxy[0].tolist()
        h, w = image_bgr.shape[:2]
        x1 = max(0, int(x1))
        y1 = max(0, int(y1))
        x2 = min(w, int(x2))
        y2 = min(h, int(y2))

        if x2 <= x1 or y2 <= y1:
            # degenerate box, fallback
            return image_bgr

        cropped = image_bgr[y1:y2, x1:x2]
        return cropped


In [9]:
def edge_density(gray_roi: np.ndarray,
                 low_thresh: int = 50,
                 high_thresh: int = 150) -> float:
    """
    Compute fraction of edge pixels in ROI using Canny.
    Returns a value in [0, 1].
    """
    if gray_roi is None or gray_roi.size == 0:
        return 0.0
    edges = cv2.Canny(gray_roi, low_thresh, high_thresh)
    nonzero = np.count_nonzero(edges)
    total = edges.size
    if total == 0:
        return 0.0
    return nonzero / total


def crop_relative(image: np.ndarray,
                  x_start: float,
                  y_start: float,
                  x_end: float,
                  y_end: float) -> np.ndarray:
    """
    Crop image using relative coordinates in [0, 1].
    (x_start, y_start) to (x_end, y_end).
    """
    h, w = image.shape[:2]
    xs = int(x_start * w)
    ys = int(y_start * h)
    xe = int(x_end * w)
    ye = int(y_end * h)
    xs = max(0, min(xs, w - 1))
    xe = max(xs + 1, min(xe, w))
    ys = max(0, min(ys, h - 1))
    ye = max(ys + 1, min(ye, h))
    return image[ys:ye, xs:xe]


def score_visibility(ed: float,
                     t_low: float = 0.08,
                     t_high: float = 0.16) -> int:
    """
    Map edge density to a 0/1/2 score:
      2 = high edges (very visible bone / structure),
      1 = medium,
      0 = low (smooth).
    """
    if ed > t_high:
        return 2
    elif ed > t_low:
        return 1
    else:
        return 0


In [10]:
def extract_side_view_features(side_bgr: np.ndarray,
                               assume_head_left: bool = True) -> SideViewFeatures:
    """
    Extract simple features from a side-view cow image.

    Assumptions:
    - side_bgr is a cropped side view of a single cow.
    - By default we assume head is on the LEFT, tail on the RIGHT.
      If your images are opposite, either set assume_head_left=False
      or horizontally flip the image before calling this function.
    """
    feats = SideViewFeatures()

    if side_bgr is None or side_bgr.size == 0:
        return feats

    # Optionally flip if head is right
    if not assume_head_left:
        side_bgr = cv2.flip(side_bgr, 1)

    gray = cv2.cvtColor(side_bgr, cv2.COLOR_BGR2GRAY)

    # --- Define ROIs in (x_start, y_start, x_end, y_end) in [0,1] ---
    # These are heuristic; you should adjust by visually inspecting crops.

    # Ribs area: mid-height, rear half
    ribs_roi = crop_relative(gray, 0.40, 0.30, 0.90, 0.75)
    ribs_ed = edge_density(ribs_roi)

    # Spine area: along top mid-body
    spine_roi = crop_relative(gray, 0.25, 0.05, 0.85, 0.30)
    spine_ed = edge_density(spine_roi)

    # Hips area: rear top quarter
    hips_roi = crop_relative(gray, 0.70, 0.10, 0.98, 0.45)
    hips_ed = edge_density(hips_roi)

    # Tail-head area: rear mid-upper zone
    tail_roi = crop_relative(gray, 0.80, 0.30, 0.98, 0.70)
    tail_ed = edge_density(tail_roi)

    # Brisket / underline area: front bottom
    brisket_roi = crop_relative(gray, 0.00, 0.60, 0.35, 1.00)
    brisket_ed = edge_density(brisket_roi)

    # --- Convert edge densities to scores ---

    # Bones: more edges → more visible → thinner
    feats.ribs_score = score_visibility(ribs_ed)
    feats.spine_score = score_visibility(spine_ed)
    feats.hips_score = score_visibility(hips_ed)

    # Tail fat: invert logic
    # - Many edges → hollows, bony tail head → low fat (score 0)
    # - Few edges  → smooth fat pads         → high fat (score 2)
    if tail_ed < 0.05:
        feats.tail_fat_score = 2
    elif tail_ed < 0.10:
        feats.tail_fat_score = 1
    else:
        feats.tail_fat_score = 0

    # Brisket fat: more structure/folds can mean more fat hanging
    # (this is crude and MUST be tuned with real photos)
    if brisket_ed > 0.18:
        feats.brisket_fat_score = 2
    elif brisket_ed > 0.10:
        feats.brisket_fat_score = 1
    else:
        feats.brisket_fat_score = 0

    return feats


In [11]:
def combine_side_features_to_bcs(
    feats: SideViewFeatures,
    alpha: float = 0.7,
    beta: float = 0.7
) -> Tuple[int, float, int, int]:
    """
    Combine thinness and fatness indices into a 1–9 BCS.

    alpha: weight for thin_index (bones)
    beta:  weight for fat_index  (fat pads)

    Returns:
        bcs (int in [1..9]),
        bcs_raw (float before rounding/clipping),
        thin_index,
        fat_index
    """

    thin_index = feats.ribs_score + feats.spine_score + feats.hips_score
    fat_index = feats.tail_fat_score + feats.brisket_fat_score

    # Start from moderate BCS 5
    bcs_raw = 5.0 - alpha * thin_index + beta * fat_index

    # Extra guards at extremes
    if thin_index >= 5 and fat_index == 0:
        bcs_raw = min(bcs_raw, 3.0)  # force very thin
    if fat_index >= 4 and thin_index == 0:
        bcs_raw = max(bcs_raw, 7.0)  # force very fat

    # Clip and round
    bcs_clipped = max(1.0, min(9.0, bcs_raw))
    bcs_final = int(round(bcs_clipped))

    return bcs_final, bcs_raw, thin_index, fat_index


In [12]:
def estimate_bcs_from_side_view(
    image_path: str,
    yolo_model_path: str = "yolov8n.pt",
    assume_head_left: bool = True
) -> Tuple[int, dict]:
    """
    Full pipeline:
      - load image
      - YOLO detect & crop cow
      - extract side view features
      - combine into BCS 1..9

    Returns:
      (bcs_score, debug_dict)
    """
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not read image at: {image_path}")

    detector = CowDetector(yolo_model_path)
    cow_crop = detector.detect_and_crop(img)

    feats = extract_side_view_features(cow_crop, assume_head_left=assume_head_left)
    bcs, bcs_raw, thin_index, fat_index = combine_side_features_to_bcs(feats)

    debug_info = {
        "features": feats,
        "bcs_raw": bcs_raw,
        "thin_index": thin_index,
        "fat_index": fat_index,
    }
    return bcs, debug_info


In [13]:
image_path = "/kaggle/input/cow-image/cow.jpg"

# Optional sanity check: image loads correctly
img = cv2.imread(image_path)
print("Image type:", type(img))
print("Image shape:", None if img is None else img.shape)

bcs_score, info = estimate_bcs_from_side_view(
    image_path,
    yolo_model_path="yolov8n.pt",   # will auto-download on first use
    assume_head_left=True           # set False if head faces right
)

print("Estimated BCS:", bcs_score)
print("Debug info:", info)


Image type: <class 'numpy.ndarray'>
Image shape: (408, 612, 3)


100%|██████████| 6.25M/6.25M [00:00<00:00, 75.8MB/s]


Estimated BCS: 4
Debug info: {'features': SideViewFeatures(ribs_score=2, spine_score=1, hips_score=1, tail_fat_score=0, brisket_fat_score=2), 'bcs_raw': 3.6, 'thin_index': 4, 'fat_index': 2}
